In [1]:
from sklearn.model_selection import train_test_split
import directory as dirt
from cv2 import cv2
import pandas as pd
import numpy as np
import os
import re

In [2]:
def mkdir_all(datapath):
    print('Generating paths: ')
    for path in datapath:
        print(path)
        if not os.path.exists(path):
            os.mkdir(path)

In [3]:
def rawdata_restore():
    '''
    @description: 二进制恢复成jpg
    '''
    raw_files = os.listdir(dirt.Dirt.raw_data_path)

    count = 0

    for raw_file in raw_files:

        f = open(dirt.Dirt.raw_data_path + "//" + raw_file)

        image = np.fromfile(f, dtype=np.uint8)

        if image.size != 262144:  # 有两张256KB的图
            image = np.reshape(image, (128, 128))
            image = cv2.resize(image, (780, 380))
        else:  # 其余的图都是16KB
            image = np.reshape(image, (512, 512))
            image = cv2.resize(image, (780, 380))

        cv2.waitKey(0)

        cv2.imwrite(dirt.Dirt.restored_path + '//' + raw_file + '.jpg', image)

        count += 1

        print('%d / %d' % (count, len(raw_files)))

In [4]:
def generateRi():
    '''
    @description: 把所有原始图片以矩阵形式存放
    '''
    raw_files = os.listdir(dirt.Dirt.raw_data_path)

    times = 0

    for raw_file in raw_files:
        with open(dirt.Dirt.raw_data_path + raw_file) as f:
            In = np.fromfile(f, dtype=np.uint8).reshape(1, -1)

            if times == 0:
                ri = In
            else:
                if ri.shape[1] != In.shape[1]:
                    In = cv2.resize(In, (ri.shape[1], 1))
                ri = np.concatenate((In, ri), axis=0)

            times += 1

            print('ri-shape:' + str(ri.shape))

    np.savetxt(dirt.ri_path, ri, fmt='%d')

    print('ri is done:' + str(ri.shape))

In [5]:
def generateIndex():
    '''
    @description: 生成图片的编号索引，并存入index文件中
    '''
    raw_files = os.listdir(dirt.Dirt.raw_data_path)
    raw_files = list(map(int, raw_files))
    raw_files = np.array(raw_files, dtype='int32')
    np.savetxt(dirt.index_path, raw_files, fmt='%d')


In [6]:
def readRiIndex():
    '''
    @description: 读取所有图片的大矩阵中矩阵的维度和索引编号
    '''
    ri = np.loadtxt(dirt.ri_path)
    print('ri import done. ri-shape: ' + str(ri.shape))

    index = np.loadtxt(dirt.index_path, dtype='int32').reshape(-1, 1)
    print('index import done. ri-shape: ' + str(index.shape))

    return ri, index

In [7]:
def generateCSV():
    '''
    @description: 把原始的全部标签文本文件转换为CSV格式
    '''
    result = pd.DataFrame(
        columns=('idx', 'sex', 'age', 'race', 'face', 'prop')
    )

    with open(dirt.faceDR_path) as f:
        lines = f.readlines()

    for line in lines:
        if not re.search('_missing', line):
            index = re.match(' ([0-9]{4})', line).group(1)
            print(index)
            sex = re.search('\(_sex  (.*?)\)', line).group(1)
            age = re.search('\(_age  (.*?)\)', line).group(1)
            race = re.search('\(_race (.*?)\)', line).group(1)
            face = re.search('\(_face (.*?)\)', line).group(1)
            prop = re.search('\(_prop \'\((.*?)\)', line).group(1)
            if not prop:
                prop = 'None'
            result = result.append(pd.DataFrame({'idx': [index], 'sex': [sex], 'age': [
                                   age], 'race': [race], 'face': [face], 'prop': [prop]}), ignore_index=True)
        else:
            continue

    with open(dirt.faceDS_path) as f:
        lines = f.readlines()
    for line in lines:
        if not re.search('_missing', line):
            index = re.match(' ([0-9]{4})', line).group(1)
            print(index)
            sex = re.search('\(_sex  (.*?)\)', line).group(1)
            age = re.search('\(_age  (.*?)\)', line).group(1)
            race = re.search('\(_race (.*?)\)', line).group(1)
            face = re.search('\(_face (.*?)\)', line).group(1)
            prop = re.search('\(_prop \'\((.*?)\)', line).group(1)
            if not prop:
                prop = 'None'
            result = result.append(pd.DataFrame({'idx': [index], 'sex': [sex], 'age': [
                                   age], 'race': [race], 'face': [face], 'prop': [prop]}), ignore_index=True)
        else:
            continue

    result.to_csv(dirt.faceD_csv_path, index=False)



In [8]:
def generateFaceRS(X, method):
    '''
    @description: 特征处理的脚本中执行该函数，执行后进行特征处理，并划分数据集，将结果写到方法所在的目录里
    '''
    y = pd.read_csv(dirt.faceD_csv_path)

    print('X-shape: ' + str(X.shape))
    print('y-shape: ' + str(y.shape))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=7
    )

    if (method == 'hog'):
        file_path = dirt.HOGFile
    elif (method == 'resnet'):
        file_path = dirt.ResnetFile

    np.savetxt(
        file_path.faceR_path, X_train,
        fmt=['%d']+['%.6f']*(X_train.shape[1]-1)
    )
    np.savetxt(
        file_path.faceS_path, X_test,
        fmt=['%d']+['%.6f']*(X_test.shape[1]-1)
    )

    y_train.to_csv(file_path.faceDR_path, index=False)
    y_test.to_csv(file_path.faceDS_path, index=False)

    print('write file to ' + file_path._path)
    print('[write] X_train.shape, X_test.shape, y_train.shape, y_test.shape:')
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



In [9]:
def readFaceRS(method):
    '''
    @description: 要用到特征处理结果的时候，执行该函数，读取特征处理后的训练、测试集结果，便于进行下一步降维/分类
    @parametre：method - 上一步生成四个文件时用的方法
    '''
    if (method == 'hog'):
        file_path = dirt.HOGFile
    elif (method == 'hog_pca'):
        file_path = dirt.HOG_PCAFile
    elif (method == 'resnet'):
        file_path = dirt.ResnetFile
    elif (method == 'resnet_kpca'):
        file_path = dirt.Resnet_KPCAFile

    # 读取数据
    X_train = np.loadtxt(file_path.faceR_path)
    X_test = np.loadtxt(file_path.faceS_path)

    y_train = pd.read_csv(file_path.faceDR_path, index_col=0)
    y_test = pd.read_csv(file_path.faceDS_path, index_col=0)

    print('read file from ' + file_path._path)
    print('[read] X_train.shape, X_test.shape, y_train.shape, y_test.shape:')
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    return X_train, X_test, y_train, y_test


In [10]:
def generateUpdateFaceRS(X_train, X_test, y_train, y_test, method):
    '''
    @description: 经过两种特征提取方法后的输出
    '''
    if (method == 'hog_pca'):
        file_path = dirt.HOG_PCAFile
    elif (method == 'resnet_kpca'):
        file_path = dirt.Resnet_KPCAFile
    np.savetxt(
        file_path.faceR_path, X_train,
        fmt=['%d']+['%.6f']*(X_train.shape[1]-1)
    )
    np.savetxt(
        file_path.faceS_path, X_test,
        fmt=['%d']+['%.6f']*(X_test.shape[1]-1)
    )

    y_train.to_csv(file_path.faceDR_path)
    y_test.to_csv(file_path.faceDS_path)

    print('write file to ' + file_path._path)
    print('[write] X_train.shape, X_test.shape, y_train.shape, y_test.shape:')
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
if __name__ == '__main__':
    directories_dict = dirt.Dirt.__dict__
    directories = [directories_dict[key]
                   for key in directories_dict if "__" not in key]

    mkdir_all(directories)

    if not os.path.exists(dirt.ri_path):
        generateRi()

    if not os.path.exists(dirt.index_path):
        generateIndex()

    if not os.path.exists(dirt.faceD_csv_path):
        generateCSV()

    # 生成原始图像，不必要一定运行
    genResFlag = True
    if not os.listdir(dirt.Dirt.restored_path) and genResFlag:
        rawdata_restore()


Generating paths: 
data/my_eigenfaces/
data/raw_data/
data/restored_data/
data/raw_label/
data/my_eigenfaces/hog/
data/my_eigenfaces/hog_pca/
data/my_eigenfaces/resnet/
data/my_eigenfaces/resnet_kpca/
image/
ri-shape:(1, 16384)
ri-shape:(2, 16384)
ri-shape:(3, 16384)
ri-shape:(4, 16384)
ri-shape:(5, 16384)
ri-shape:(6, 16384)
ri-shape:(7, 16384)
ri-shape:(8, 16384)
ri-shape:(9, 16384)
ri-shape:(10, 16384)
ri-shape:(11, 16384)
ri-shape:(12, 16384)
ri-shape:(13, 16384)
ri-shape:(14, 16384)
ri-shape:(15, 16384)
ri-shape:(16, 16384)
ri-shape:(17, 16384)
ri-shape:(18, 16384)
ri-shape:(19, 16384)
ri-shape:(20, 16384)
ri-shape:(21, 16384)
ri-shape:(22, 16384)
ri-shape:(23, 16384)
ri-shape:(24, 16384)
ri-shape:(25, 16384)
ri-shape:(26, 16384)
ri-shape:(27, 16384)
ri-shape:(28, 16384)
ri-shape:(29, 16384)
ri-shape:(30, 16384)
ri-shape:(31, 16384)
ri-shape:(32, 16384)
ri-shape:(33, 16384)
ri-shape:(34, 16384)
ri-shape:(35, 16384)
ri-shape:(36, 16384)
ri-shape:(37, 16384)
ri-shape:(38, 16384)
ri-

ri-shape:(376, 16384)
ri-shape:(377, 16384)
ri-shape:(378, 16384)
ri-shape:(379, 16384)
ri-shape:(380, 16384)
ri-shape:(381, 16384)
ri-shape:(382, 16384)
ri-shape:(383, 16384)
ri-shape:(384, 16384)
ri-shape:(385, 16384)
ri-shape:(386, 16384)
ri-shape:(387, 16384)
ri-shape:(388, 16384)
ri-shape:(389, 16384)
ri-shape:(390, 16384)
ri-shape:(391, 16384)
ri-shape:(392, 16384)
ri-shape:(393, 16384)
ri-shape:(394, 16384)
ri-shape:(395, 16384)
ri-shape:(396, 16384)
ri-shape:(397, 16384)
ri-shape:(398, 16384)
ri-shape:(399, 16384)
ri-shape:(400, 16384)
ri-shape:(401, 16384)
ri-shape:(402, 16384)
ri-shape:(403, 16384)
ri-shape:(404, 16384)
ri-shape:(405, 16384)
ri-shape:(406, 16384)
ri-shape:(407, 16384)
ri-shape:(408, 16384)
ri-shape:(409, 16384)
ri-shape:(410, 16384)
ri-shape:(411, 16384)
ri-shape:(412, 16384)
ri-shape:(413, 16384)
ri-shape:(414, 16384)
ri-shape:(415, 16384)
ri-shape:(416, 16384)
ri-shape:(417, 16384)
ri-shape:(418, 16384)
ri-shape:(419, 16384)
ri-shape:(420, 16384)
ri-shape:(

ri-shape:(764, 16384)
ri-shape:(765, 16384)
ri-shape:(766, 16384)
ri-shape:(767, 16384)
ri-shape:(768, 16384)
ri-shape:(769, 16384)
ri-shape:(770, 16384)
ri-shape:(771, 16384)
ri-shape:(772, 16384)
ri-shape:(773, 16384)
ri-shape:(774, 16384)
ri-shape:(775, 16384)
ri-shape:(776, 16384)
ri-shape:(777, 16384)
ri-shape:(778, 16384)
ri-shape:(779, 16384)
ri-shape:(780, 16384)
ri-shape:(781, 16384)
ri-shape:(782, 16384)
ri-shape:(783, 16384)
ri-shape:(784, 16384)
ri-shape:(785, 16384)
ri-shape:(786, 16384)
ri-shape:(787, 16384)
ri-shape:(788, 16384)
ri-shape:(789, 16384)
ri-shape:(790, 16384)
ri-shape:(791, 16384)
ri-shape:(792, 16384)
ri-shape:(793, 16384)
ri-shape:(794, 16384)
ri-shape:(795, 16384)
ri-shape:(796, 16384)
ri-shape:(797, 16384)
ri-shape:(798, 16384)
ri-shape:(799, 16384)
ri-shape:(800, 16384)
ri-shape:(801, 16384)
ri-shape:(802, 16384)
ri-shape:(803, 16384)
ri-shape:(804, 16384)
ri-shape:(805, 16384)
ri-shape:(806, 16384)
ri-shape:(807, 16384)
ri-shape:(808, 16384)
ri-shape:(

ri-shape:(1146, 16384)
ri-shape:(1147, 16384)
ri-shape:(1148, 16384)
ri-shape:(1149, 16384)
ri-shape:(1150, 16384)
ri-shape:(1151, 16384)
ri-shape:(1152, 16384)
ri-shape:(1153, 16384)
ri-shape:(1154, 16384)
ri-shape:(1155, 16384)
ri-shape:(1156, 16384)
ri-shape:(1157, 16384)
ri-shape:(1158, 16384)
ri-shape:(1159, 16384)
ri-shape:(1160, 16384)
ri-shape:(1161, 16384)
ri-shape:(1162, 16384)
ri-shape:(1163, 16384)
ri-shape:(1164, 16384)
ri-shape:(1165, 16384)
ri-shape:(1166, 16384)
ri-shape:(1167, 16384)
ri-shape:(1168, 16384)
ri-shape:(1169, 16384)
ri-shape:(1170, 16384)
ri-shape:(1171, 16384)
ri-shape:(1172, 16384)
ri-shape:(1173, 16384)
ri-shape:(1174, 16384)
ri-shape:(1175, 16384)
ri-shape:(1176, 16384)
ri-shape:(1177, 16384)
ri-shape:(1178, 16384)
ri-shape:(1179, 16384)
ri-shape:(1180, 16384)
ri-shape:(1181, 16384)
ri-shape:(1182, 16384)
ri-shape:(1183, 16384)
ri-shape:(1184, 16384)
ri-shape:(1185, 16384)
ri-shape:(1186, 16384)
ri-shape:(1187, 16384)
ri-shape:(1188, 16384)
ri-shape:(1

ri-shape:(1508, 16384)
ri-shape:(1509, 16384)
ri-shape:(1510, 16384)
ri-shape:(1511, 16384)
ri-shape:(1512, 16384)
ri-shape:(1513, 16384)
ri-shape:(1514, 16384)
ri-shape:(1515, 16384)
ri-shape:(1516, 16384)
ri-shape:(1517, 16384)
ri-shape:(1518, 16384)
ri-shape:(1519, 16384)
ri-shape:(1520, 16384)
ri-shape:(1521, 16384)
ri-shape:(1522, 16384)
ri-shape:(1523, 16384)
ri-shape:(1524, 16384)
ri-shape:(1525, 16384)
ri-shape:(1526, 16384)
ri-shape:(1527, 16384)
ri-shape:(1528, 16384)
ri-shape:(1529, 16384)
ri-shape:(1530, 16384)
ri-shape:(1531, 16384)
ri-shape:(1532, 16384)
ri-shape:(1533, 16384)
ri-shape:(1534, 16384)
ri-shape:(1535, 16384)
ri-shape:(1536, 16384)
ri-shape:(1537, 16384)
ri-shape:(1538, 16384)
ri-shape:(1539, 16384)
ri-shape:(1540, 16384)
ri-shape:(1541, 16384)
ri-shape:(1542, 16384)
ri-shape:(1543, 16384)
ri-shape:(1544, 16384)
ri-shape:(1545, 16384)
ri-shape:(1546, 16384)
ri-shape:(1547, 16384)
ri-shape:(1548, 16384)
ri-shape:(1549, 16384)
ri-shape:(1550, 16384)
ri-shape:(1

...

694 / 3993
695 / 3993
696 / 3993
697 / 3993
698 / 3993
699 / 3993
700 / 3993
701 / 3993
702 / 3993
703 / 3993
704 / 3993
705 / 3993
706 / 3993
707 / 3993
708 / 3993
709 / 3993
710 / 3993
711 / 3993
712 / 3993
713 / 3993
714 / 3993
715 / 3993
716 / 3993
717 / 3993
718 / 3993
719 / 3993
720 / 3993
721 / 3993
722 / 3993
723 / 3993
724 / 3993
725 / 3993
726 / 3993
727 / 3993
728 / 3993
729 / 3993
730 / 3993
731 / 3993
732 / 3993
733 / 3993
734 / 3993
735 / 3993
736 / 3993
737 / 3993
738 / 3993
739 / 3993
740 / 3993
741 / 3993
742 / 3993
743 / 3993
744 / 3993
745 / 3993
746 / 3993
747 / 3993
748 / 3993
749 / 3993
750 / 3993
751 / 3993
752 / 3993
753 / 3993
754 / 3993
755 / 3993
756 / 3993
757 / 3993
758 / 3993
759 / 3993
760 / 3993
761 / 3993
762 / 3993
763 / 3993
764 / 3993
765 / 3993
766 / 3993
767 / 3993
768 / 3993
769 / 3993
770 / 3993
771 / 3993
772 / 3993
773 / 3993
774 / 3993
775 / 3993
776 / 3993
777 / 3993
778 / 3993
779 / 3993
780 / 3993
781 / 3993
782 / 3993
783 / 3993
784 / 3993

1427 / 3993
1428 / 3993
1429 / 3993
1430 / 3993
1431 / 3993
1432 / 3993
1433 / 3993
1434 / 3993
1435 / 3993
1436 / 3993
1437 / 3993
1438 / 3993
1439 / 3993
1440 / 3993
1441 / 3993
1442 / 3993
1443 / 3993
1444 / 3993
1445 / 3993
1446 / 3993
1447 / 3993
1448 / 3993
1449 / 3993
1450 / 3993
1451 / 3993
1452 / 3993
1453 / 3993
1454 / 3993
1455 / 3993
1456 / 3993
1457 / 3993
1458 / 3993
1459 / 3993
1460 / 3993
1461 / 3993
1462 / 3993
1463 / 3993
1464 / 3993
1465 / 3993
1466 / 3993
1467 / 3993
1468 / 3993
1469 / 3993
1470 / 3993
1471 / 3993
1472 / 3993
1473 / 3993
1474 / 3993
1475 / 3993
1476 / 3993
1477 / 3993
1478 / 3993
1479 / 3993
1480 / 3993
1481 / 3993
1482 / 3993
1483 / 3993
1484 / 3993
1485 / 3993
1486 / 3993
1487 / 3993
1488 / 3993
1489 / 3993
1490 / 3993
1491 / 3993
1492 / 3993
1493 / 3993
1494 / 3993
1495 / 3993
1496 / 3993
1497 / 3993
1498 / 3993
1499 / 3993
1500 / 3993
1501 / 3993
1502 / 3993
1503 / 3993
1504 / 3993
1505 / 3993
1506 / 3993
1507 / 3993
1508 / 3993
1509 / 3993
1510

2138 / 3993
2139 / 3993
2140 / 3993
2141 / 3993
2142 / 3993
2143 / 3993
2144 / 3993
2145 / 3993
2146 / 3993
2147 / 3993
2148 / 3993
2149 / 3993
2150 / 3993
2151 / 3993
2152 / 3993
2153 / 3993
2154 / 3993
2155 / 3993
2156 / 3993
2157 / 3993
2158 / 3993
2159 / 3993
2160 / 3993
2161 / 3993
2162 / 3993
2163 / 3993
2164 / 3993
2165 / 3993
2166 / 3993
2167 / 3993
2168 / 3993
2169 / 3993
2170 / 3993
2171 / 3993
2172 / 3993
2173 / 3993
2174 / 3993
2175 / 3993
2176 / 3993
2177 / 3993
2178 / 3993
2179 / 3993
2180 / 3993
2181 / 3993
2182 / 3993
2183 / 3993
2184 / 3993
2185 / 3993
2186 / 3993
2187 / 3993
2188 / 3993
2189 / 3993
2190 / 3993
2191 / 3993
2192 / 3993
2193 / 3993
2194 / 3993
2195 / 3993
2196 / 3993
2197 / 3993
2198 / 3993
2199 / 3993
2200 / 3993
2201 / 3993
2202 / 3993
2203 / 3993
2204 / 3993
2205 / 3993
2206 / 3993
2207 / 3993
2208 / 3993
2209 / 3993
2210 / 3993
2211 / 3993
2212 / 3993
2213 / 3993
2214 / 3993
2215 / 3993
2216 / 3993
2217 / 3993
2218 / 3993
2219 / 3993
2220 / 3993
2221

2886 / 3993
2887 / 3993
2888 / 3993
2889 / 3993
2890 / 3993
2891 / 3993
2892 / 3993
2893 / 3993
2894 / 3993
2895 / 3993
2896 / 3993
2897 / 3993
2898 / 3993
2899 / 3993
2900 / 3993
2901 / 3993
2902 / 3993
2903 / 3993
2904 / 3993
2905 / 3993
2906 / 3993
2907 / 3993
2908 / 3993
2909 / 3993
2910 / 3993
2911 / 3993
2912 / 3993
2913 / 3993
2914 / 3993
2915 / 3993
2916 / 3993
2917 / 3993
2918 / 3993
2919 / 3993
2920 / 3993
2921 / 3993
2922 / 3993
2923 / 3993
2924 / 3993
2925 / 3993
2926 / 3993
2927 / 3993
2928 / 3993
2929 / 3993
2930 / 3993
2931 / 3993
2932 / 3993
2933 / 3993
2934 / 3993
2935 / 3993
2936 / 3993
2937 / 3993
2938 / 3993
2939 / 3993
2940 / 3993
2941 / 3993
2942 / 3993
2943 / 3993
2944 / 3993
2945 / 3993
2946 / 3993
2947 / 3993
2948 / 3993
2949 / 3993
2950 / 3993
2951 / 3993
2952 / 3993
2953 / 3993
2954 / 3993
2955 / 3993
2956 / 3993
2957 / 3993
2958 / 3993
2959 / 3993
2960 / 3993
2961 / 3993
2962 / 3993
2963 / 3993
2964 / 3993
2965 / 3993
2966 / 3993
2967 / 3993
2968 / 3993
2969

3618 / 3993
3619 / 3993
3620 / 3993
3621 / 3993
3622 / 3993
3623 / 3993
3624 / 3993
3625 / 3993
3626 / 3993
3627 / 3993
3628 / 3993
3629 / 3993
3630 / 3993
3631 / 3993
3632 / 3993
3633 / 3993
3634 / 3993
3635 / 3993
3636 / 3993
3637 / 3993
3638 / 3993
3639 / 3993
3640 / 3993
3641 / 3993
3642 / 3993
3643 / 3993
3644 / 3993
3645 / 3993
3646 / 3993
3647 / 3993
3648 / 3993
3649 / 3993
3650 / 3993
3651 / 3993
3652 / 3993
3653 / 3993
3654 / 3993
3655 / 3993
3656 / 3993
3657 / 3993
3658 / 3993
3659 / 3993
3660 / 3993
3661 / 3993
3662 / 3993
3663 / 3993
3664 / 3993
3665 / 3993
3666 / 3993
3667 / 3993
3668 / 3993
3669 / 3993
3670 / 3993
3671 / 3993
3672 / 3993
3673 / 3993
3674 / 3993
3675 / 3993
3676 / 3993
3677 / 3993
3678 / 3993
3679 / 3993
3680 / 3993
3681 / 3993
3682 / 3993
3683 / 3993
3684 / 3993
3685 / 3993
3686 / 3993
3687 / 3993
3688 / 3993
3689 / 3993
3690 / 3993
3691 / 3993
3692 / 3993
3693 / 3993
3694 / 3993
3695 / 3993
3696 / 3993
3697 / 3993
3698 / 3993
3699 / 3993
3700 / 3993
3701